In [ ]:

# Library Management System

class Library:
    """
    A library class for managing a collection of books.

    Attributes:
        _books (list): Private list of books in the library.

    Examples:
        >>> lib = Library()
        >>> lib.add_book("1984", "George Orwell", "9780451524935", 1949, "Dystopian")
        >>> len(lib.books)
        1
        >>> lib.search_books("orwell")[0]['title']
        '1984'
    """

    def __init__(self):
        """Initialize an empty library."""
        self._books = []

    @property
    def books(self):
        """Return a copy of the books list."""
        return self._books.copy()

    def add_book(self, title, author, isbn, year, genre):
        """
        Add a book to the library with validation.

        Args:
            title (str): Book title
            author (str): Book author
            isbn (str): Book ISBN
            year (int): Publication year
            genre (str): Book genre

        Raises:
            ValueError: If any parameter is invalid or ISBN already exists.
        """
        if not all(isinstance(param, str) and param.strip() for param in [title, author, isbn, genre]):
            raise ValueError("Title, author, ISBN, and genre must be non-empty strings.")
        if not isinstance(year, int) or not (1000 <= year <= 2100):
            raise ValueError("Year must be an integer between 1000 and 2100.")
        if any(b['isbn'] == isbn for b in self._books):
            raise ValueError(f"Book with ISBN {isbn} already exists.")

        self._books.append({
            "title": title.strip(),
            "author": author.strip(),
            "isbn": isbn.strip(),
            "year": year,
            "genre": genre.strip(),
            "available": True,
            "rating": None
        })

    def search_books(self, query):
        """Return books where query matches title or author (case-insensitive)."""
        query = query.lower()
        return [b for b in self._books if query in b['title'].lower() or query in b['author'].lower()]

    def get_book(self, isbn):
        """Return book by ISBN or None if not found."""
        for b in self._books:
            if b['isbn'] == isbn:
                return b
        return None

    def __str__(self):
        return f"Library with {len(self._books)} books"

    def __repr__(self):
        return f"Library(books={len(self._books)})"


class Library2:
    """
    Handles checkouts and availability for a Library instance.

    Attributes:
        _library (Library): Library object
        _checkouts (list): List of checkout records
        _users (list): List of users who checked out books

    Examples:
        >>> lib = Library()
        >>> lib.add_book("1984", "George Orwell", "9780451524935", 1949, "Dystopian")
        >>> manager = Library2(lib)
        >>> manager.checkout_book("Alice", "9780451524935")
        'Alice checked out 1984.'
    """

    def __init__(self, library):
        if not isinstance(library, Library):
            raise TypeError("library must be a Library instance")
        self._library = library
        self._checkouts = []
        self._users = []

    @property
    def checkouts(self):
        """Return a copy of checkout records."""
        return self._checkouts.copy()

    @property
    def users(self):
        """Return a copy of users list."""
        return self._users.copy()

    def checkout_book(self, user_name, isbn):
        """
        Check out a book to a user if available.

        Args:
            user_name (str): Name of user
            isbn (str): Book ISBN

        Returns:
            str: Confirmation message
        """
        book = self._library.get_book(isbn)
        if not book:
            return "Book not found."
        if not book['available']:
            return "Book is already checked out."

        book['available'] = False
        self._checkouts.append({"user": user_name, "isbn": isbn})
        if user_name not in self._users:
            self._users.append(user_name)
        return f"{user_name} checked out {book['title']}."

    def return_book(self, user_name, isbn, days_late=0, fee_per_day=0.25):
        """
        Return a book and calculate late fee.

        Args:
            user_name (str): Name of user
            isbn (str): Book ISBN
            days_late (int): Number of days late
            fee_per_day (float): Fee per day late

        Returns:
            str: Confirmation message with late fee
        """
        book = self._library.get_book(isbn)
        if not book:
            return "Book not found."
        if book['available']:
            return "Book was not checked out."

        book['available'] = True
        self._checkouts = [c for c in self._checkouts if not (c['user'] == user_name and c['isbn'] == isbn)]
        fee = max(days_late * fee_per_day, 0)
        return f"{user_name} returned {book['title']}. Late fee: ${fee:.2f}"


class LibraryUtils:
    """
    Utility class for library operations like late fee and genre search.

    Examples:
        >>> utils = LibraryUtils()
        >>> utils.calculate_late_fee(3)
        0.75
    """

    @staticmethod
    def calculate_late_fee(days_late, fee_per_day=0.25):
        """Return late fee for given days."""
        if days_late < 0:
            return 0
        return days_late * fee_per_day

    @staticmethod
    def find_books_by_genre(genre, book_list):
        """Return books matching genre."""
        return [b for b in book_list if b['genre'].lower() == genre.lower()]

    @staticmethod
    def find_book_by_isbn(isbn, book_list):
        """Return book by ISBN."""
        for b in book_list:
            if b['isbn'] == isbn:
                return b
        return None


class LibraryStats:
    """
    Analyze books, movies, and user activity.

    Attributes:
        _library (list): Books list
        _movies (list): Movies list
        _users (list): Users list

    Examples:
        >>> stats = LibraryStats(library=[{"title": "Book A", "rating":5}], movies=[], users=[])
        >>> stats.get_highest_rated_books()
        ['Book A']
    """

    def __init__(self, library=None, movies=None, users=None):
        self._library = library if library is not None else []
        self._movies = movies if movies is not None else []
        self._users = users if users is not None else []

    @property
    def library(self):
        return self._library

    @property
    def movies(self):
        return self._movies

    @property
    def users(self):
        return self._users

    def get_highest_rated_books(self):
        rated = [b for b in self._library if b.get("rating") is not None]
        if not rated:
            return []
        max_rating = max(b["rating"] for b in rated)
        return [b["title"] for b in rated if b["rating"] == max_rating]

    def get_unrated_books(self):
        return [b["title"] for b in self._library if b.get("rating") is None]

    def get_highest_rated_movies(self):
        rated = [m for m in self._movies if m.get("rating") is not None]
        if not rated:
            return []
        max_rating = max(m["rating"] for m in rated)
        return [m["title"] for m in rated if m["rating"] == max_rating]

    def get_unrated_movies(self):
        return [m["title"] for m in self._movies if m.get("rating") is None]

    def get_top_users(self):
        stats = []
        for u in self._users:
            total = u.get("books_read", 0) + u.get("movies_watched", 0)
            stats.append((u.get("name", "Unknown"), total))
        if not stats:
            return []
        max_total = max(total for _, total in stats)
        return [(name, total) for name, total in stats if total == max_total]

    def __str__(self):
        return f"LibraryStats with {len(self._library)} books, {len(self._movies)} movies, and {len(self._users)} users."

    def __repr__(self):
        return f"LibraryStats(library={len(self._library)}, movies={len(self._movies)}, users={len(self._users)})"


# -------------------------------
# Sample / Test Data
# -------------------------------
if __name__ == "__main__":
    # Initialize Library
    lib = Library()
    lib.add_book("1984", "George Orwell", "9780451524935", 1949, "Dystopian")
    lib.add_book("Animal Farm", "George Orwell", "9780451526342", 1945, "Satire")
    lib.add_book("To Kill a Mockingbird", "Harper Lee", "9780061120084", 1960, "Classic")

    print(lib)
    print(lib.books)

    # Initialize Checkout Manager
    manager = Library2(lib)
    print(manager.checkout_book("Alice", "9780451524935"))
    print(manager.checkout_book("Bob", "9780451526342"))
    print(manager.return_book("Alice", "9780451524935", days_late=2))

    # Utilities
    utils = LibraryUtils()
    print("Late fee for 3 days:", utils.calculate_late_fee(3))
    print("Dystopian books:", utils.find_books_by_genre("Dystopian", lib.books))

    # Stats
    users = [
        {"name": "Alice", "books_read": 1, "movies_watched": 0},
        {"name": "Bob", "books_read": 1, "movies_watched": 1}
    ]
    stats = LibraryStats(library=lib.books, users=users)
    print(stats.get_top_users())
    print(stats)


Library with 3 books
[{'title': '1984', 'author': 'George Orwell', 'isbn': '9780451524935', 'year': 1949, 'genre': 'Dystopian', 'available': True, 'rating': None}, {'title': 'Animal Farm', 'author': 'George Orwell', 'isbn': '9780451526342', 'year': 1945, 'genre': 'Satire', 'available': True, 'rating': None}, {'title': 'To Kill a Mockingbird', 'author': 'Harper Lee', 'isbn': '9780061120084', 'year': 1960, 'genre': 'Classic', 'available': True, 'rating': None}]
Alice checked out 1984.
Bob checked out Animal Farm.
Alice returned 1984. Late fee: $0.50
Late fee for 3 days: 0.75
Dystopian books: [{'title': '1984', 'author': 'George Orwell', 'isbn': '9780451524935', 'year': 1949, 'genre': 'Dystopian', 'available': True, 'rating': None}]
[('Bob', 2)]
LibraryStats with 3 books, 0 movies, and 2 users.
